# ４章

### word2vecの改良①

In [1]:
#必要なライブラリのインポート
import sys,os
sys.path.append('/zero-tuku2-2024/common/')
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

３章で扱ったモデルは小さなコーパス（１文章）のみで考えました。
しかし、通常は非常に大きなコーパスを扱うことになります。

![](picture/pict2.png)

語彙数が100万のCBOWモデルの場合計算量が莫大で

>・重み行列Wの積の計算（in,out)<br>
 ・sofutmaxレイヤでの計算<br>

これらがボトルネックになってしまう。<br>また、入力層のone-hot表現は無駄な要素が多くなることも明らか。

### Embedding レイヤ


入力層では以下の計算がされていた。

![](picture/pict3.png)

ここで行われているのは1つの単語に対応する行列の行を抜き出しているだけ。これを簡略化するために<br>

> 「単語IDに該当する行（ベクトル）」を抜き出すためのレイヤ

を作成しよう。 ここではそのレイヤをEmbeddingと呼ぶことにする。


### Embedding レイヤ の実装

行列から特定の行を抜き出すには、以下のように行う。

In [2]:
W = np.arange(21).reshape(7,3)
print(W) 
print()
print(W[2]) #2行目を取り出す
print()
print(W[5]) #5行目を取り出す

[[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]
 [12 13 14]
 [15 16 17]
 [18 19 20]]

[6 7 8]

[15 16 17]


In [3]:
idx = np.array([1,0,3,0])
W[idx] # 1行目、0行目、3行目、0行目を抽出

array([[ 3,  4,  5],
       [ 0,  1,  2],
       [ 9, 10, 11],
       [ 0,  1,  2]])

EmbeddingレイヤのForward()メソッドを実装

In [4]:
class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None

    def forward(self, idx):
        W, = self.params
        self.idx = idx # idxを抽出する行のインデックスとして保持
        out = W[idx]
        return out

次は、逆伝播についても考える。
![](picture/pict4.png)

\begin{aligned}
\vec{h} &: (1, 3) \\
\vec{W} &: (L, 3) \text{とすると} \\
\vec{W} &= \begin{pmatrix}
\vec{W_1} \\
\vdots \\
\vec{W_L}
\end{pmatrix} \text{ で, forward で流れた } \vec{h} \text{ について, } \\
\vec{h} &= \vec{W_k} \quad (1 \leq k \leq L) \\
\text{よって, } \quad d\vec{h} &= d\vec{W_k}
\end{aligned}

これを踏まえて実装してみる。


In [5]:
def backward(self, dout):
        dW, = self.grads # dWは重みの勾配
        dW[...] = 0 # dWをゼロで初期化
        dW[self.idx] = dout # 実は悪い例
        return None

### **注意**
自分たちの目的は行列Wの更新なので、いちいちdWのような行列を作る必要はない。

必要な部品は<br>
>・更新したい行番号（idx）<br>
>・その勾配（dout)<br>

これらを保持すれば、重みWの特定の行のみを更新することができる。

しかし、このbackwardには問題点がある。
![](picture/pict5.png)

dhの各行の値をidxで指定された場所へ代入すると、dWの０番目に２つの値が代入されてしまいどちらかの値が上書きされてしまう。この問題は加算して代入することが必要。（理由は板書で説明）

正しい逆伝播の実装は次のようになる。

In [6]:
#逆伝播の修正
def backward(self, dout):
    dW, = self.grads
    dW[...] = 0
    np.add.at(dW, self.idx, dout)
    
    return None

以上によって入力層の重み行列の問題が解決された！<br>
### word2vecの改良②

![](picture/pict2.png)

残りの問題は
> 中間層の重み行列の積の計算量<br>
>出力層ソフトマックスの計算量<br>

これらを解決するNegative samplingについて説明していく。
### 多値分類から二値分類へ


結論からいうと、Negative samplingのキーになるアイデアは「**二値分類**」にあります。正確には、「多値分類」を「二値分類」で近似することです。<br>

今までは、NNに対して、<br>
「**コンテキストが『you』と『goodbye』のとき、ターゲットとなる単語はなんですか？**」と質問して学習を行っていました。<br>

これを「yes」「no」で答えられる質問に置き換えます。

例えば<br>
「**コンテキストが『you』と『goodbye』のとき、ターゲットとなる単語は『say』ですか？**」という質問に答えるNNを考えることにすると、ニューロンを一つのみ用意すればよいことになります。


![](picture/pict6.png)

中間層と出力側の重み行列の積は、「say」に対応する列ベクトルだけを取り出し、その抽出したベクトルと中間層のニューロンの内積を取れば良い。

![](picture/pict7.png)

出力側の重みWoutには各単語IDの単語ベクトルが各列に並んで格納されています。ここでは「say」という単語ベクトルを抽出します。

### シグモイド関数と交差エントロピー誤差

「多値分類」から「二値分類」に変えたので、スコアを出力する関数をソフトマックスからシグモイド関数へ変えます。（シグモイド関数はソフトマックスのN＝2のとき）<br>損失関数はそのまま交差エントロピーにします。

##### シグモイド関数

$$
\sigma(x) = \frac{1}{1 + e^{-x}}
$$

##### 交差エントロピー誤差

$$
L = -\left[ t \log(y) + (1 - t) \log(1 - y) \right]
$$

※　tは正解ラベルで、yはシグモイド関数で計算された確率


![](picture/pict8.png)

### 復習　Sigmoid with Loss レイヤ
![](picture/pict9.png)

まず、シグモイド関数の微分は以下の通り
$$
\text{Sigmoid} \quad f(x) = \frac{1}{1 + e^{-x}} = \frac{e^x}{e^x + 1}
$$

$$
f'(x) = \frac{e^x(e^x + 1) - e^x \cdot e^x}{(e^x + 1)^2} = \frac{e^x}{(e^x + 1)^2}
$$

$$
= \frac{e^x}{e^x + 1} \cdot \frac{1}{e^x + 1} = f(x) \left(1 - f(x)\right)
$$


興味があるのは、xが少し変化したときLはどれくらい変わるのかということ

$$
\frac{\partial L}{\partial x} = \frac{\partial L}{\partial y} \cdot \frac{dy}{dx}
$$

$$
= \left( -\frac{t}{y} - \frac{1-t}{1-y} \right) \cdot y(1-y)
$$

$$
= \frac{(1-t)y - (1-y)t}{y(1-y)} \cdot y(1-y) = y - t
$$

これらの議論から二値分類を行うNNは以下のようになります。

![](picture/pict10.png)

入力層と出力層の重み行列の計算は対応するベクトルを抜き出すことで計算量を落とせました。（Embedding）見通しをよくするため、後半部分をよりシンプルにします。

![](picture/pict11.png)

W_outのEmbeddingと中間層の内積をまとめたEmbedding Dotレイヤを作ります。

In [7]:
class EmbeddingDot:
    def __init__(self, W):
        self.embed = Embedding(W) # Embeddingレイヤ
        self.params = self.embed.params # パラメータ
        self.grads = self.embed.grads  #v勾配
        self.cache = None # 順伝播の入力値を保持

    def forward(self, h, idx):
        target_W = self.embed.forward(idx)
        out = np.sum(target_W * h, axis=1)

        self.cache = (h, target_W)
        return out

    def backward(self, dout):
        h, target_W = self.embed.params 
        dout = dout.reshape(dout.shape[0], 1) # 3つの要素のリストを(3,1)に変換し、ブロードキャストできるようにする

        dtarget_W = dout * h #(3,1) * (3,7) = (3,7)
        self.embed.backward(dtarget_W)
        dh = dout * target_W
        return dh

In [8]:
#forwardの処理解説
W = np.arange(21).reshape(7,3)
idx = np.array([0,3,1])
h = np.array([[0,1,2],[3,4,5],[6,7,8],]) #中間層のニューロン
embed = Embedding(W)
target_W = embed.forward(idx)
out = np.sum(target_W * h, axis=1)
#W、idx,target_W,h,target_W * h, outを表にして出力
print("W")
print(W)
print("idx")
print(idx)
print("target_W")
print(target_W)
print("h")
print(h)
print("target_W * h")
print(target_W * h)
print("out")
print(out) # (1,3)の行列

W
[[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]
 [12 13 14]
 [15 16 17]
 [18 19 20]]
idx
[0 3 1]
target_W
[[ 0  1  2]
 [ 9 10 11]
 [ 3  4  5]]
h
[[0 1 2]
 [3 4 5]
 [6 7 8]]
target_W * h
[[ 0  1  4]
 [27 40 55]
 [18 28 40]]
out
[  5 122  86]


In [9]:
#backwardの処理解説
dout = np.array([1,2,3])
dout = dout.reshape(dout.shape[0], 1)
dtarget_W = dout * h
print("dout")
print(dout)
print("h")
print(h)
print("dout * h")
print(dout * h)
print("dtarget_W")
print(dtarget_W)
dh = dout * target_W
print("dh")
print(dh)

dout
[[1]
 [2]
 [3]]
h
[[0 1 2]
 [3 4 5]
 [6 7 8]]
dout * h
[[ 0  1  2]
 [ 6  8 10]
 [18 21 24]]
dtarget_W
[[ 0  1  2]
 [ 6  8 10]
 [18 21 24]]
dh
[[ 0  1  2]
 [18 20 22]
 [ 9 12 15]]


### Negative sampling

これまで、正例（正しい答え）についてのみ学習を行っていました。<br>

そのため、負例（誤った答え）については、どうなるか定かではありません。<br>

私達は、NNが「いい分類」をしているか、「悪い分類」をしているか教えてあげる必要があります。つまり、「悪い例」を与えることで精度を効率的に上げることを考えます。

![](picture/pict12.png)

では、「悪い例」を教えることにしますが、これは全ての負例について行っていては、計算量を減らすという目的が達成できません。

そこで「悪い例」を少数サンプリングして用いることにします。<br>

この手法を「***Negative Sampling***」といいます。

この手法では、正例と負例についてそれぞれで損失を求め、その総和を最終的な損失として学習をします。

### Negative Samplingのサンプリング手法

サンプリングはどのように行うのでしょうか。完全にランダムにサンプルするよりも、良い方法が知られています。

それはコーパス内の単語の使用頻度に基づいてサンプリングする方法です。

この使用頻度に応じたサンプリングにはnp.random.choice()関数を使います。

In [10]:
#random.choiceの使い方
np.random.choice(10)

3

In [11]:
#wordsの中からランダムに一つ選ぶ
words=['you','say','goodbye','I','hello','.']
np.random.choice(words)

'I'

In [12]:
#5つだけランダムに選ぶ（重複あり）
np.random.choice(words, size=5)

array(['goodbye', 'say', 'hello', 'goodbye', 'hello'], dtype='<U7')

In [13]:
#5つだけランダムに選ぶ（重複なし）
np.random.choice(words, size=5, replace=False)

array(['you', 'say', 'hello', 'I', 'goodbye'], dtype='<U7')

In [14]:
#確率分布に従ってサンプリング
p=[0.5, 0.1, 0.05, 0.2, 0.05, 0.1]
np.random.choice(words, p=p)

'.'

random.choice()の引数は以下の３つです<br>
size：サンプリングの回数指定<br>
replace：重複の有無（デフォルトはTrue）<br>
p：確率分布を指定

word2vecでは与える確率分布にも一手間加えています。

それは確率分布の小数乗を行うことです。

この操作の意図は出現確率の低い単語の確率を少しだけ上げ、それらの単語を見捨てないようにするためです。

$$
P'(w_i) = \frac{P(w_i)^{0.75}}{\sum_{j=1}^{n} P(w_j)^{0.75}}
$$

In [15]:
p=[0.7, 0.29, 0.01]
new_p=np.power(p, 0.75) #累乗する関数np.power
new_p /= np.sum(new_p)
print(new_p)

[0.64196878 0.33150408 0.02652714]


これらの操作をUnigramSamplerクラスとしてまとめます

In [16]:
import sys
sys.path.append('..')
#UnigramSamplerの実装
import collections
from common import config
class UnigramSampler:
    def __init__(self, corpus, power, sample_size):
        self.sample_size = sample_size
        self.vocab_size = None
        self.word_p = None

        counts = collections.Counter()
        for word_id in corpus:
            counts[word_id] += 1

        vocab_size = len(counts)
        self.vocab_size = vocab_size

        self.word_p = np.zeros(vocab_size)
        for i in range(vocab_size):
            self.word_p[i] = counts[i]

        self.word_p = np.power(self.word_p, power)
        self.word_p /= np.sum(self.word_p)

    def get_negative_sample(self, target):
        batch_size = target.shape[0]

        #if not GPU: GPU is not defined というエラーが出るので、コメントアウトしました
        negative_sample = np.zeros((batch_size, self.sample_size), dtype=np.int32)

        for i in range(batch_size):
                p = self.word_p.copy()
                target_idx = target[i]
                p[target_idx] = 0
                p /= p.sum()
                negative_sample[i, :] = np.random.choice(self.vocab_size, size=self.sample_size, replace=False, p=p)
        #else:
            # GPU(cupy）で計算するときは、速度を優先
            # 負例にターゲットが含まれるケースがある
         #   negative_sample = np.random.choice(self.vocab_size, size=(batch_size, self.sample_size),
         #                                     replace=True, p=self.word_p)

        return negative_sample

In [17]:
#UnigramSamplerのサンプリング
import numpy as np 
import collections

corpus = np.array([0, 1, 2, 3, 4, 1, 2, 3])
power = 0.75
sample_size = 2

sampler = UnigramSampler(corpus, power, sample_size)
target = np.array([1, 3, 0])
negative_sample = sampler.get_negative_sample(target)
print(negative_sample)

[[3 0]
 [2 1]
 [3 2]]


### Negative Sampling の実装

In [18]:
#NegativeSamplingLossの実装
class NegativeSamplingLoss:
    def __init__(self, W, corpus, power=0.75, sample_size=5):
        self.sample_size = sample_size #負例のサンプリング数
        self.sampler = UnigramSampler(corpus, power, sample_size) #UnigramSamplerの保持
        self.loss_layers = [SigmoidWithLoss() for _ in range(sample_size + 1)] #SigmoidWithLossレイヤの保持
        self.embed_dot_layers = [EmbeddingDot(W) for _ in range(sample_size + 1)] #EmbeddingDotレイヤの保持

        #lossレイヤとembed_dotレイヤは負例の数＋正例(sample_size+1)個分用意する

        self.params, self.grads = [], []
        for layer in self.embed_dot_layers:
            self.params += layer.params
            self.grads += layer.grads

    def forward(self, h, target): #引数は中間層のニューロンhとターゲットの単語ID
        batch_size = target.shape[0] #バッチサイズ
        negative_sample = self.sampler.get_negative_sample(target) #負例のサンプリング

        # 正例のフォワード
        score = self.embed_dot_layers[0].forward(h, target)
        correct_label = np.ones(batch_size, dtype=np.int32) #正例の正解ラベルは1
        loss = self.loss_layers[0].forward(score, correct_label)

        # 負例のフォワード
        negative_label = np.zeros(batch_size, dtype=np.int32) #負例の正解ラベルは0
        for i in range(self.sample_size): #負例の数だけループ
            negative_target = negative_sample[:, i]
            score = self.embed_dot_layers[1 + i].forward(h, negative_target)
            loss += self.loss_layers[1 + i].forward(score, negative_label) #負例の損失を加算
        
        #lossには正例の損失と負例の損失が合算されている

        return loss

    def backward(self, dout=1):
        dh = 0
        for l0, l1 in zip(self.loss_layers, self.embed_dot_layers):
            dscore = l0.backward(dout)
            dh += l1.backward(dscore)

        return dh

### 改良版word2vecの学習

CBOWモデルの実装

この章ではEmbeddingレイヤとNegative sampling loss レイヤを紹介してきました。

これらの改良点を踏まえて前章のSimpleCBOWクラスを改良します。

In [19]:
#改良版CBOWモデルの実装
import sys
sys.path.append('..')
from common.np import *  # import numpy as np
from common.layers import Embedding
#from ch04.negative_sampling_layer import NegativeSamplingLoss


class CBOW:
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size

        # 重みの初期化
        W_in = 0.01 * np.random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(V, H).astype('f')

        # レイヤの生成
        self.in_layers = []
        for i in range(2 * window_size):
            layer = Embedding(W_in)  # Embeddingレイヤを使用
            self.in_layers.append(layer)
        self.ns_loss = NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5)

        # すべての重みと勾配をリストにまとめる
        layers = self.in_layers + [self.ns_loss]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads

        # メンバ変数に単語の分散表現を設定
        self.word_vecs = W_in

    def forward(self, contexts, target):
        h = 0
        for i, layer in enumerate(self.in_layers):
            h += layer.forward(contexts[:, i])
        h *= 1 / len(self.in_layers)
        loss = self.ns_loss.forward(h, target)
        return loss

    def backward(self, dout=1):
        dout = self.ns_loss.backward(dout)
        dout *= 1 / len(self.in_layers)
        for layer in self.in_layers:
            layer.backward(dout)
        return None